# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [0]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [0]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
import numpy as np

In [0]:
# try this project here!
import random
Q = 23740629843760239486723

def encrypt(x, n_shares = 3):
  shares = list()
  # We generate the first 2 values randomly
  # and use them to generate the 3rd
  for i in range(n_shares -1):
    shares.append(random.randint(0,Q))
  # sum_shares % Q (a replace value within mod Q)
  final_share = Q - (sum(shares) % Q) + x
  # print(final_share)
  # print(Q - (sum(shares)% Q))
  # print(sum(shares) % Q)
  # print((sum(shares)% Q) + x)

  shares.append(final_share)
  # print(sum(shares)%Q)
  return tuple(shares)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [0]:
def add(a, b):
   c = list()
   assert(len(a) == len(b))
   for i in range(len(a)):
     c.append((a[i]+b[i])% Q)

   return tuple(c)

In [9]:
decrypt(add(encrypt(10), encrypt(4)))

14

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [11]:
encode(3.5)

35000

In [0]:
decode(35000)

3.5

In [12]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
# secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice)

In [35]:
bob._objects

{20990274906: tensor([1291164013452428696, 1425430391468367224,  961361942061240478,
         3066154825066784066, 1734320051308343156])}

In [42]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:81649078867 -> bob:36196711599]
	-> [PointerTensor | me:21161574256 -> alice:69451597740]
	*crypto provider: me*

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [37]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:42348630637 -> bob:38040110800]
	-> [PointerTensor | me:58060063825 -> alice:74144605723]
	*crypto provider: me*

In [38]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [0]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [4]:
x = x.fix_prec().share(bob, alice, secure_worker)

NameError: ignored

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [0]:
from collections import OrderedDict 

In [2]:
import torch as th
import torchvision.datasets as datasets
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),])
mnist_trn = datasets.MNIST(root='./data_', train=True, download=True, transform=transform)
mnist_val = datasets.MNIST(root='./data_', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

9920512it [00:01, 8760776.98it/s]                            


Extracting ./data_/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 143145.60it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data_/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2424839.69it/s]                           
0it [00:00, ?it/s]

Extracting ./data_/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 52953.11it/s]            

Extracting ./data_/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [3]:
!pip install syft
import syft as sy
hook  = sy.TorchHook(th)

     |████████████████████████████████| 286kB 5.0MB/s 
     |████████████████████████████████| 204kB 40.1MB/s 
     |████████████████████████████████| 256kB 26.5MB/s 
     |████████████████████████████████| 81kB 28.5MB/s 
     |████████████████████████████████| 460kB 36.2MB/s 
     |████████████████████████████████| 389kB 40.7MB/s 
     |████████████████████████████████| 2.1MB 42.1MB/s 
     |████████████████████████████████| 51kB 20.2MB/s 
     |████████████████████████████████| 266kB 45.6MB/s 
     |████████████████████████████████| 122kB 44.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067080 sha256=6d5e2bff0f04283286cf796a2ac00c0d701dfbaf91ba7d2358bd210e8ecdaabd
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=5e9be53c4c6ac08371a5c89aa63ea93800fc65d958ed361d8ce81c14547a4259
  Stored in di

W0824 09:16:38.023785 140414890219392 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0824 09:16:38.043019 140414890219392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# Hyperparameters
bs = 5
classes = 10
input_dim = 784
H = 256
epochs = 10

In [0]:
m = th.nn.Sequential(OrderedDict({
        'ln_1': th.nn.Linear(input_dim, H),
        'ReLU_1': th.nn.ReLU(),
        'ln_2': th.nn.Linear(H,H),
        'ReLU_2':th.nn.ReLU(),
        'ln_2':th.nn.Linear(H,classes),
        'log_softmax':th.nn.LogSoftmax(dim=1)}))

In [0]:
bob = sy.VirtualWorker(hook, id ="bob")
alice = sy.VirtualWorker(hook, id="alice")
jacob = sy.VirtualWorker(hook, id ="jacob")
bob = bob.clear_objects()
alice = alice.clear_objects()
jacob = jacob.clear_objects()

In [0]:
# data splits
trn_bob = sy.BaseDataset(mnist_trn.data[0:300], mnist_trn.targets[0:300]).send(bob)
trn_alice = sy.BaseDataset(mnist_trn.data[300:600],mnist_trn.targets[300:600]).send(alice)
trn_jacob = sy.BaseDataset(mnist_trn.data[600:900],mnist_trn.targets[600:900]).send(jacob)


val_bob = sy.BaseDataset(mnist_val.data[0:50], mnist_val.targets[0:50]).send(bob)
val_alice = sy.BaseDataset(mnist_val.data[50:100],mnist_val.data[50:100]).send(alice)
val_jacob = sy.BaseDataset(mnist_val.data[50:100],mnist_val.data[50:100]).send(jacob)


# Creating federated datasets, an extension of Pytorch TensorDataset class
federated_train_dataset = sy.FederatedDataset([trn_bob, trn_alice, trn_jacob])
federated_test_dataset = sy.FederatedDataset([val_bob, val_alice, val_jacob])


# Creating federated dataloaders, an extension of Pytorch DataLoader class
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, 
                                                shuffle=True, batch_size=bs)
federated_test_loader = sy.FederatedDataLoader(federated_test_dataset, 
                                               shuffle=False, batch_size=bs)

In [0]:
crit_bob = th.nn.NLLLoss()
crit_alice = th.nn.NLLLoss()
crit_jacob = th.nn.NLLLoss()
opt_bob = th.optim.SGD(m.parameters(), lr = 1e-2) 
opt_alice = th.optim.SGD(m.parameters(), lr = 1e-4)
opt_jacob = th.optim.SGD(m.parameters(), lr = 1e-3)

In [0]:
def unit_test(jacob, model):  
    a = model.copy().send(jacob)
    weights = set_ws_bs(model,a, a, a)
    
    # a.get()
    print("Org model Ln_1 weights", a.ln_1.weight.data)
    print("Org model Ln_1 updated weights", weights.ln_1.weight.data)
    del a, weights

In [0]:
unit_test(jacob, m)

In [0]:
import pdb

In [0]:
def get_ws_bs(model_bob,model_alice, model_jacob):
    avg_ws = {}
    avg_bs = {}
    for (_, m_1),(i, m_2), (j, m_3) in zip(model_bob.named_children(), model_alice.named_children(), model_jacob.named_children()):
        if(i != 'ReLU_1' and i != 'ReLU_2' and i != 'log_softmax'):
            # pdb.set_trace()
            # fix_prec -> encrypt shares -> add -> decode -> update
            bob_ws = m_1.weight.data
            bob_ws = bob_ws.fix_prec().share(model_bob.location.id, model_alice.location.id, model_jacob.location.id)
            bob_ws = bob_ws.get()
            # bob_ws > bob > ASS
            # we retrieve the child the pointer to bob which is the ASS shares
            # thats why call get() method twice
            # bob_ws > ASS

            bob_bs = m_1.bias.data
            bob_bs = bob_bs.fix_prec().share(model_bob.location.id, model_alice.location.id, model_jacob.location.id)
            bob_bs = bob_bs.get()

            alice_ws = m_2.weight.data
            alice_ws = alice_ws.fix_prec().share(model_bob.location.id, model_alice.location.id, model_jacob.location.id)
            alice_ws = alice_ws.get()

            alice_bs = m_2.bias.data
            alice_bs = alice_bs.fix_prec().share(model_bob.location.id, model_alice.location.id, model_jacob.location.id)
            alice_bs = alice_bs.get()

            jacob_ws = m_3.weight.data
            jacob_ws = jacob_ws.fix_prec().share(model_bob.location.id, model_alice.location.id, model_jacob.location.id)
            jacob_ws = jacob_ws.get()

            jacob_bs = m_3.bias.data
            jacob_bs = jacob_bs.fix_prec().share(model_bob.location.id, model_alice.location.id, model_jacob.location.id)
            jacob_bs = jacob_bs.get()

            # pdb.set_trace()
            avg_ws[i] = ((bob_ws + alice_ws + jacob_ws)/3).get().float_prec()
            avg_bs[i] = ((bob_bs + alice_bs + jacob_bs)/3).get().float_prec()

    return avg_ws, avg_bs

def set_ws_bs(model, model_bob, model_alice, model_jacob):
    avg_ws, avg_bs = get_ws_bs(model_bob, model_alice, model_jacob)
    with th.no_grad():
        for i, layer in model.named_children():
          if(i != 'ReLU_1' and i != 'ReLU_2' and i != 'log_softmax'):
              layer.weight.data = avg_ws[i]
              layer.bias.data = avg_bs[i]
    
    return model

In [0]:
import matplotlib.pyplot as plt
def fit(model, bob, alice, jacob,iterations =10):
    model.train()
    # pdb.set_trace()
    model_bob = model.copy().send(bob)
    model_alice = model.copy().send(alice)
    model_jacob = model.copy().send(jacob)

    for j in range(iterations):
        for i, (x, y) in enumerate(federated_train_loader):

            x = x.view(x.shape[0],-1)
            # do normal training
            if (x.location.id == 'bob'):
                # model_bob = model.copy().send(x.location.id)
                opt_bob.zero_grad()
                pred_bob = model_bob(x.float())
                loss_bob = crit_bob(pred_bob, y)
                loss_bob.backward()
                opt_bob.step() 
                if i % 10 == 0:
                    print("Bob's_loss: ", str(loss_bob.get().data))
                
                      
            if (x.location.id == 'alice'):
                # model_alice = model.copy().send(x.location.id)
                opt_alice.zero_grad()
                pred_alice = model_alice(x.float())
                loss_alice = crit_alice(pred_alice, y)
                loss_alice.backward()
                opt_alice.step()
                if i % 10 == 0:
                    print("Alice's_loss: ", str(loss_alice.get().data))

            if (x.location.id == 'jacob'):
                opt_jacob.zero_grad()
                pred_jacob = model_jacob(x.float())
                loss_jacob = crit_jacob(pred_jacob, y)
                loss_jacob.backward()
                opt_alice.step()
                if i % 5 == 0:
                    print("Jacob's_loss", str(loss_jacob.get().data))
                
            # get smarter model back

    model = set_ws_bs(model ,model_bob, model_alice, model_jacob)

    
    return model  

In [21]:
model_1 = fit(m, bob, alice, jacob)

Bob's_loss:  tensor(14.7972)
Bob's_loss:  tensor(20.3779)
Bob's_loss:  tensor(15.3838)
Bob's_loss:  tensor(15.3418)
Bob's_loss:  tensor(13.8235)
Bob's_loss:  tensor(4.8220)
Alice's_loss:  tensor(10.0980)
Alice's_loss:  tensor(10.6476)
Alice's_loss:  tensor(16.1031)
Alice's_loss:  tensor(14.5205)
Alice's_loss:  tensor(16.2195)
Alice's_loss:  tensor(10.4959)
Jacob's_loss tensor(13.4014)
Jacob's_loss tensor(11.5075)
Jacob's_loss tensor(24.0016)
Jacob's_loss tensor(12.6722)
Jacob's_loss tensor(22.7061)
Jacob's_loss tensor(14.2173)
Jacob's_loss tensor(7.8946)
Jacob's_loss tensor(14.6120)
Jacob's_loss tensor(16.0634)
Jacob's_loss tensor(12.6834)
Jacob's_loss tensor(16.7588)
Jacob's_loss tensor(17.5612)
Bob's_loss:  tensor(11.2718)
Bob's_loss:  tensor(24.2666)
Bob's_loss:  tensor(17.1943)
Bob's_loss:  tensor(13.9822)
Bob's_loss:  tensor(15.8913)
Bob's_loss:  tensor(15.7963)
Alice's_loss:  tensor(20.5913)
Alice's_loss:  tensor(20.2676)
Alice's_loss:  tensor(10.8573)
Alice's_loss:  tensor(10.07

In [24]:
print(m.ln_1.weight.data)
print(model_1.ln_1.weight.data)

tensor([[-0.0319,  0.0153,  0.0121,  ..., -0.0190, -0.0026, -0.0027],
        [ 0.0303, -0.0003,  0.0217,  ...,  0.0177,  0.0043, -0.0356],
        [ 0.0346, -0.0008,  0.0178,  ..., -0.0139,  0.0130,  0.0274],
        ...,
        [-0.0078, -0.0217,  0.0284,  ...,  0.0322,  0.0050,  0.0062],
        [-0.0290, -0.0301,  0.0211,  ..., -0.0104, -0.0003,  0.0207],
        [ 0.0154, -0.0278, -0.0289,  ..., -0.0201, -0.0139, -0.0321]])
tensor([[ 2.8000e-02, -3.5000e-02,  5.0000e-03,  ...,  2.4000e-02,
         -1.5372e+15, -1.5372e+15],
        [-1.1000e-02,  2.9000e-02, -1.5372e+15,  ...,  1.9000e-02,
         -9.0000e-03, -1.7000e-02],
        [-1.5372e+15, -3.4000e-02,  1.5000e-02,  ..., -1.5372e+15,
         -1.5372e+15,  3.1000e-02],
        ...,
        [-1.5372e+15, -2.2000e-02, -1.5372e+15,  ...,  5.0000e-03,
         -1.5372e+15,  3.1000e-02],
        [-1.5000e-02, -2.2000e-02, -1.0000e-03,  ...,  1.0000e-02,
          2.2000e-02, -2.6000e-02],
        [ 3.0000e-02, -1.5372e+15, -1.